<a href="https://colab.research.google.com/github/Jar199/2022-1-Euron-Study-Assignments/blob/Week_2/euron_w2_netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

추천 시스템 공부 관련 코드1 : https://www.kaggle.com/chocozzz/t-academy-recommendation/code  
추천 시스템 공부 관련 코드2: https://www.kaggle.com/chocozzz/t-academy-recommendation2/code  
필사할 넷플릭스 추천 시스템 원본 코드: https://www.kaggle.com/eward96/netflix-recommendation-engine  

#Intro  
netflix에서 방영되는 영화와 TV쇼를 추천하는 코드, 두가지 방법을 비교해 볼것  
- 배우, 감독, 국가, 평점과 장르를 feature로 사용  
- 영화 및 TV쇼 서술의 단어들을 feature로 사용

In [1]:
# Import
import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Data 불러오기
data = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
print(data.head())
print("=================================================")
print(data.groupby('type').count())
print("=================================================")
data = data.dropna(subset=['cast', 'country', 'rating'])

  show_id     type  title           director  \
0      s1  TV Show     3%                NaN   
1      s2    Movie   7:19  Jorge Michel Grau   
2      s3    Movie  23:59       Gilbert Chan   
3      s4    Movie      9        Shane Acker   
4      s5    Movie     21     Robert Luketic   

                                                cast        country  \
0  João Miguel, Bianca Comparato, Michel Gomes, R...         Brazil   
1  Demián Bichir, Héctor Bonilla, Oscar Serrano, ...         Mexico   
2  Tedd Chan, Stella Chung, Henley Hii, Lawrence ...      Singapore   
3  Elijah Wood, John C. Reilly, Jennifer Connelly...  United States   
4  Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...  United States   

          date_added  release_year rating   duration  \
0    August 14, 2020          2020  TV-MA  4 Seasons   
1  December 23, 2016          2016  TV-MA     93 min   
2  December 20, 2018          2011      R     78 min   
3  November 16, 2017          2009  PG-13     80 min   
4   

# 배우, 감독, 국가, 평점 및 장르를 사용하여 추천 엔진을 개발

In [6]:
# 영화 관련
movies = data[data['type']=='Movie'].reset_index()
movies = movies.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
movies.head()
print() 
# tv쇼 관련
tv = data[data['type']=='TV Show'].reset_index()
t = tv.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
tv.head() 

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,5,s6,TV Show,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,"July 1, 2017",2016,TV-MA,1 Season,"International TV Shows, TV Dramas, TV Mysteries",A genetics professor experiments with a treatm...
2,11,s12,TV Show,1983,NaN,"Robert Więckiewicz, Maciej Musiał, Michalina O...","Poland, United States","November 30, 2018",2018,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Dramas","In this dark alt-history thriller, a naïve law..."
3,24,s25,TV Show,​SAINT SEIYA: Knights of the Zodiac,NaN,"Bryson Baugus, Emily Neves, Blake Shepard, Pat...",Japan,"January 23, 2020",2020,TV-14,2 Seasons,"Anime Series, International TV Shows",Seiya and the Knights of the Zodiac rise again...
4,29,s30,TV Show,#blackAF,NaN,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,"April 17, 2020",2020,TV-MA,1 Season,TV Comedies,Kenya Barris and his family navigate relations...


In [7]:
# 영화 관련 -> one-hot 만들기
actors = []

for i in movies['cast']:
  actor = re.split(r', \s*', i) # 하나의 영화 안에서도 여러명의 cast존재 -> 각각 쪼개 각각 하나의 actor로
  actors.append(actor)

flat_list = [] 
# 여러개의 vector로 되어있는 actor 이름들을  -> list로 나타냄
for sublist in actors:
  for item in sublist:
    flat_list.append(item)

# 배우들 list
actors_list = sorted(set(flat_list)) # 중복 없애고 배우 이름 순으로 나열한 list

binary_actors = [[0]*0 for i in range(len(set(flat_list)))]
# (?)x(후보 actor 수)

# movies['cast']에 저장되어있는 배우 순으로 본인 이름이면 1추가, 본인 이름 아니면 0추가 해서 0혹은 1로 구성된 matrix 생성(본인 이름에 표시된)
for i in movies['cast']:
  k = 0
  for j in actors_list:
    if j in i:
      binary_actors[k].append(1.0) # 나온 배우들에 1로 표기
    else:
      binary_actors[k].append(0.0)
    k +=1
# (movie 수)x(후보 actor 수)

binary_actors = pd.DataFrame(binary_actors).transpose()
# (후보 actor 수)x(movie 수)

directors = []

for i in movies['director']:
  if pd.notna(i): # 누락값 여부 체크하는 함수
    director = re.split(r', \s*', i)
    directors.append(director)

flat_list2 = []
for sublist in directors:
  for item in sublist:
    flat_list2.append(item)

directors_list = sorted(set(flat_list2))

binary_directors = [[0]*0 for i in range(len(set(flat_list2)))]

for i in movies['director']:
  k=0
  for j in directors_list:
    if pd.isna(i):
      binary_directors[k].append(0.0)
    elif j in i: # null값도 아니고 directors_list에 있는 경우에만 1
      binary_directors[k].append(1.0)
    else:
      binary_directors[k].append(0.0)
    k+=1

binary_directors = pd.DataFrame(binary_directors).transpose()

countries = []

for i in movies['country']:
    country = re.split(r', \s*', i)
    countries.append(country)
    
flat_list3 = []
for sublist in countries:
    for item in sublist:
        flat_list3.append(item)
        
countries_list = sorted(set(flat_list3))

binary_countries = [[0] * 0 for i in range(len(set(flat_list3)))]

for i in movies['country']:
    k = 0
    for j in countries_list:
        if j in i:
            binary_countries[k].append(1.0)
        else:
            binary_countries[k].append(0.0)
        k+=1
        
binary_countries = pd.DataFrame(binary_countries).transpose()

genres = []

for i in movies['listed_in']:
    genre = re.split(r', \s*', i)
    genres.append(genre)
    
flat_list4 = []
for sublist in genres:
    for item in sublist:
        flat_list4.append(item)
        
genres_list = sorted(set(flat_list4))

binary_genres = [[0] * 0 for i in range(len(set(flat_list4)))]

for i in movies['listed_in']:
    k = 0
    for j in genres_list:
        if j in i:
            binary_genres[k].append(1.0)
        else:
            binary_genres[k].append(0.0)
        k+=1
        
binary_genres = pd.DataFrame(binary_genres).transpose()

ratings = []

for i in movies['rating']:
    ratings.append(i)

ratings_list = sorted(set(ratings))

binary_ratings = [[0] * 0 for i in range(len(set(ratings_list)))]

for i in movies['rating']:
    k = 0
    for j in ratings_list:
        if j in i:
            binary_ratings[k].append(1.0)
        else:
            binary_ratings[k].append(0.0)
        k+=1
        
binary_ratings = pd.DataFrame(binary_ratings).transpose()

In [9]:
binary = pd.concat([binary_actors, binary_directors, binary_countries, binary_genres], axis=1, ignore_index=True)
print(binary)
print(np.shape(binary_actors))
print(np.shape(binary_directors))
print(np.shape(binary_countries))
print(np.shape(binary_genres))
# 종류의 수가 가장 적은 cast에서 중복 제거한 cast의 종류의 수가 row개수가 되고 

      0      1      2      3      4      5      6      7      8      9      \
0       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
4756    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4757    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4758    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4759    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4760    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

      ...  26570  26571  26572  26573  26574  26575  26576  265

In [8]:
# TV 쇼의 table 만들기
actors2 = []

for i in tv['cast']:
    actor2 = re.split(r', \s*', i)
    actors2.append(actor2)
    
flat_list5 = []
for sublist in actors2:
    for item in sublist:
        flat_list5.append(item)
        
actors_list2 = sorted(set(flat_list5))

binary_actors2 = [[0] * 0 for i in range(len(set(flat_list5)))]

for i in tv['cast']:
    k = 0
    for j in actors_list2:
        if j in i:
            binary_actors2[k].append(1.0)
        else:
            binary_actors2[k].append(0.0)
        k+=1
        
binary_actors2 = pd.DataFrame(binary_actors2).transpose()
        

countries2 = []

for i in tv['country']:
    country2 = re.split(r', \s*', i)
    countries2.append(country2)
    
flat_list6 = []
for sublist in countries2:
    for item in sublist:
        flat_list6.append(item)
        
countries_list2 = sorted(set(flat_list6))

binary_countries2 = [[0] * 0 for i in range(len(set(flat_list6)))]

for i in tv['country']:
    k = 0
    for j in countries_list2:
        if j in i:
            binary_countries2[k].append(1.0)
        else:
            binary_countries2[k].append(0.0)
        k+=1
        
binary_countries2 = pd.DataFrame(binary_countries2).transpose()

genres2 = []

for i in tv['listed_in']:
    genre2 = re.split(r', \s*', i)
    genres2.append(genre2)
    
flat_list7 = []
for sublist in genres2:
    for item in sublist:
        flat_list7.append(item)
        
genres_list2 = sorted(set(flat_list7))

binary_genres2 = [[0] * 0 for i in range(len(set(flat_list7)))]

for i in tv['listed_in']:
    k = 0
    for j in genres_list2:
        if j in i:
            binary_genres2[k].append(1.0)
        else:
            binary_genres2[k].append(0.0)
        k+=1
        
binary_genres2 = pd.DataFrame(binary_genres2).transpose()

ratings2 = []

for i in tv['rating']:
    ratings2.append(i)

ratings_list2 = sorted(set(ratings2))

binary_ratings2 = [[0] * 0 for i in range(len(set(ratings_list2)))]

for i in tv['rating']:
    k = 0
    for j in ratings_list2:
        if j in i:
            binary_ratings2[k].append(1.0)
        else:
            binary_ratings2[k].append(0.0)
        k+=1
        
binary_ratings2 = pd.DataFrame(binary_ratings2).transpose()

In [10]:
binary2 = pd.concat([binary_actors2, binary_countries2, binary_genres2], axis=1, ignore_index=True)
print(binary2)

      0      1      2      3      4      5      6      7      8      9      \
0       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
1886    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1887    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1888    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1889    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1890    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

      ...  12741  12742  12743  12744  12745  12746  12747  127

##영화 및 TV 쇼의 배우, 감독, 국가, 평점 정보 사용하여 추천 시스템 구현

In [11]:
def recommender(search):
    cs_list = []
    binary_list = []
    # 사용자가 입력한 영화 혹은 tv 쇼의 제목에 해당하는 영화를 찾아 배우, 감독, 국가, 평점의 정보를 찾아낸다.
    ## 영화인지 확인
    if search in movies['title'].values:
        idx = movies[movies['title'] == search].index.item()
        for i in binary.iloc[idx]: # 영화의 정보들을 list에 넣는다
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies)):
            binary_list2 = []
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            # 두 영화(입력한 영화와 모든 영화 중 하나)의 유사도를 계산
            ## 수식의 의미?
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search] # 동일한 영화는 제외
        top_results = results.head(5) # 유사도 가장 높은 5개의 영화 출력
        return(top_results)
    ## tv 쇼인지 확인
    elif search in tv['title'].values:
        idx = tv[tv['title'] == search].index.item()
        for i in binary2.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in binary2.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

## 영화를 추천해본다  
영화 제목을 입력하면 해당 영화와 비슷한 영화들을 추천해준다.

In [13]:
print(recommender('The Conjuring'))

KeyboardInterrupt: ignored

In [ ]:
print(recommender("Child's Play"))

In [ ]:
print(recommender('Charlie and the Chocolate Factory'))

In [ ]:
print(recommender('Wild Child'))

In [ ]:
print(recommender("Dr. Seuss' The Cat in the Hat"))

In [ ]:
print(recommender('Holidate'))

## TV 쇼를 추천해본다.  
방법은 영화 추천과 같음.

In [ ]:
print(recommender('After Life'))

In [ ]:
print(recommender('Anne with an E'))

In [ ]:
print(recommender('Gilmore Girls'))

In [ ]:
print(recommender('Love on the Spectrum'))

In [ ]:
print(recommender('13 Reasons Why'))

In [ ]:
print(recommender('Derry Girls'))

In [ ]:
print(recommender('Breaking Bad'))

# 영화 및 TV 쇼의 설명을 이용한 추천 엔진을 개발

In [ ]:
# 영화
movies_des = data[data['type'] == 'Movie'].reset_index()
movies_des = movies_des[['title', 'description']]
print(movies_des.head())

In [ ]:
# TV 쇼
tv_des = data[data['type'] == 'TV Show'].reset_index()
tv_des = tv_des[['title', 'description']]
print(tv_des.head())

In [ ]:
filtered_movies = []
movies_words = []

# 설명 문장을 토큰 단어들의 list로 만들고 단어들을 모두 소문자로만 구성되도록 처리한다.
## 영화
for text in movies_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    movies_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_movies.append(filtered)

movies_words = [val for sublist in movies_words for val in sublist]
movies_words = sorted(set(movies_words))
movies_des['description_filtered'] = filtered_movies
print(movies_des.head())

In [ ]:
## tv 쇼
filtered_tv = []
tv_words = []

for text in tv_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    tv_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_tv.append(filtered)

tv_words = [val for sublist in tv_words for val in sublist]
tv_words = sorted(set(tv_words))
tv_des['description_filtered'] = filtered_tv
print(tv_des.head())

In [ ]:
# 첫번째 방법에서 한 것처럼 해당 단어들의 포함 유무로 table을 생성
## 영화
movie_word_binary = [[0] * 0 for i in range(len(set(movies_words)))]

for des in movies_des['description_filtered']:
    k = 0
    for word in movies_words:
        if word in des:
            movie_word_binary[k].append(1.0)
        else:
            movie_word_binary[k].append(0.0)
        k+=1
        
movie_word_binary = pd.DataFrame(movie_word_binary).transpose()

## tv 쇼
tv_word_binary = [[0] * 0 for i in range(len(set(tv_words)))]

for des in tv_des['description_filtered']:
    k = 0
    for word in tv_words:
        if word in des:
            tv_word_binary[k].append(1.0)
        else:
            tv_word_binary[k].append(0.0)
        k+=1
        
tv_word_binary = pd.DataFrame(tv_word_binary).transpose()

## 영화 및 tv 쇼에 대한 설명의 데이터를 사용하여 추천 시스템을 구현한다.

In [ ]:
# 다루는 list만 다를 뿐 전체적인 내용은 recommander()와 거의 똑같음
def recommender2(search):
    cs_list = []
    binary_list = []
    if search in movies_des['title'].values:
        idx = movies_des[movies_des['title'] == search].index.item()
        for i in movie_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies_des)):
            binary_list2 = []
            for k in movie_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies_des.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    elif search in tv_des['title'].values:
        idx = tv_des[tv_des['title'] == search].index.item()
        for i in tv_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in tv_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv_des.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

## 영화를 추천해본다.  
영화 제목을 검색하면 해당 영화와 설명이 비슷한 영화들을 추천해준다.

In [ ]:
pd.options.display.max_colwidth = 300
print(recommender2('The Conjuring'))

In [ ]:
print(recommender2("Child's Play"))

In [ ]:
print(recommender2('Charlie and the Chocolate Factory'))

In [ ]:
print(recommender2('Wild Child'))

In [ ]:
print(recommender2("Dr. Seuss' The Cat in the Hat"))

In [ ]:
print(recommender2('Holidate'))

## TV 쇼를 추천해본다.  
방법은 영화 추천과 같음.

In [ ]:
print(recommender2('After Life'))

In [ ]:
print(recommender2('Anne with an E'))

In [ ]:
print(recommender2('Gilmore Girls'))

In [ ]:
print(recommender2('Love on the Spectrum'))

In [ ]:
print(recommender2('13 Reasons Why'))

In [ ]:
print(recommender2('Derry Girls'))

In [ ]:
print(recommender2('Breaking Bad'))